Source: https://torchgeo.readthedocs.io/en/stable/tutorials/getting_started.html

Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Getting Started

In this tutorial, we demonstrate some of the basic features of TorchGeo and show how easy it is to use if you're already familiar with other PyTorch domain libraries like torchvision.

It's recommended to run this notebook on Google Colab if you don't have your own GPU. Click the "Open in Colab" button above to get started.

## Setup

You need a Python environment to run this notebook. You can either do this in Google Colab, or by setting up a local python environment. I like to set my up using conda:

In [ ]:
# create a conda environment with python 3.11 for pytorch
%conda create --name pyt2 python=3.11
%conda activate pyt2

# run this next line specific to your OS and GPU configuration. Use CPU if you don't have CUDA.
# https://pytorch.org/get-started/locally/
# this example is for a linux system with CUDA 12.1
%conda install pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia

Now we can install TorchGeo. It's best to use `pip` for this.

In [3]:
%pip install torchgeo

  Using cached rasterio-1.4.0-cp311-cp311-win_amd64.whl.metadata (9.3 kB)
Using cached rasterio-1.4.0-cp311-cp311-win_amd64.whl (25.3 MB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
# for this tutorial, we need to manually downgrade rasterio to an earlier version to get around a bug
%pip install "rasterio<1.4"

  Using cached rasterio-1.3.11-cp311-cp311-win_amd64.whl.metadata (15 kB)
Using cached rasterio-1.3.11-cp311-cp311-win_amd64.whl (24.8 MB)
  Attempting uninstall: rasterio
    Found existing installation: rasterio 1.4.0
    Uninstalling rasterio-1.4.0:
      Successfully uninstalled rasterio-1.4.0
Note: you may need to restart the kernel to use updated packages.


## Imports

Next, we import TorchGeo and any other libraries we need.

In [6]:
import os
import tempfile

from torch.utils.data import DataLoader

from torchgeo.datasets import NAIP, ChesapeakeDE, stack_samples
from torchgeo.datasets.utils import download_url
from torchgeo.samplers import RandomGeoSampler

## Datasets

For this tutorial, we'll be using imagery from the [National Agriculture Imagery Program (NAIP)](https://catalog.data.gov/dataset/national-agriculture-imagery-program-naip) and labels from the [Chesapeake Bay High-Resolution Land Cover Project](https://www.chesapeakeconservancy.org/conservation-innovation-center/high-resolution-data/land-cover-data-project/). First, we manually download a few NAIP tiles and create a PyTorch Dataset.

In [7]:
naip_root = os.path.join(tempfile.gettempdir(), 'naip')
naip_url = (
    'https://naipeuwest.blob.core.windows.net/naip/v002/de/2018/de_060cm_2018/38075/'
)
tiles = [
    'm_3807511_ne_18_060_20181104.tif',
    'm_3807511_se_18_060_20181104.tif',
    'm_3807512_nw_18_060_20180815.tif',
    'm_3807512_sw_18_060_20180815.tif',
]
for tile in tiles:
    download_url(naip_url + tile, naip_root)

naip = NAIP(naip_root)

Using downloaded and verified file: C:\Users\ilyon\AppData\Local\Temp\naip\m_3807511_ne_18_060_20181104.tif
Using downloaded and verified file: C:\Users\ilyon\AppData\Local\Temp\naip\m_3807511_se_18_060_20181104.tif
Using downloaded and verified file: C:\Users\ilyon\AppData\Local\Temp\naip\m_3807512_nw_18_060_20180815.tif
Using downloaded and verified file: C:\Users\ilyon\AppData\Local\Temp\naip\m_3807512_sw_18_060_20180815.tif


Next, we tell TorchGeo to automatically download the corresponding Chesapeake labels.

In [8]:
chesapeake_root = os.path.join(tempfile.gettempdir(), 'chesapeake')
os.makedirs(chesapeake_root, exist_ok=True)
chesapeake = ChesapeakeDE(chesapeake_root, crs=naip.crs, res=naip.res, download=True)

Finally, we create an IntersectionDataset so that we can automatically sample from both GeoDatasets simultaneously.

In [9]:
dataset = naip & chesapeake

## Sampler

Unlike typical PyTorch Datasets, TorchGeo GeoDatasets are indexed using lat/long/time bounding boxes. This requires us to use a custom GeoSampler instead of the default sampler/batch_sampler that comes with PyTorch.

In [10]:
sampler = RandomGeoSampler(dataset, size=1000, length=10)

## DataLoader

Now that we have a Dataset and Sampler, we can combine these into a single DataLoader.

In [11]:
dataloader = DataLoader(dataset, sampler=sampler, collate_fn=stack_samples)

## Training

Other than that, the rest of the training pipeline is the same as it is for torchvision.

In [12]:
for sample in dataloader:
    image = sample['image']
    target = sample['mask']